In [ ]:
import dataframeLoader as dfl
import pandas as pd

from importlib import reload 
reload(dfl)

root = '.dataDir'
appliance_id='396254dc-3d6c-45e8-8a2b-0fd8d956a704'
fromDt = '2024-09-26'
toDt = '2024-10-03'

# metricsArr = ['cpu_used', 'download_workers_count', 'memory_used', 'task_queue_length', 'infra_access_latency', 'pod_cpu_usage', 'pod_memory_usage'] 
metricsArr = ['cpu_used','task_queue_length', 'memory_used']
daterange=[fromDt, toDt]

dfsp = dfl.loadConnectorDataFromFileRegex(root, 'SCANPROC-*.csv', daterange=daterange)
dfst = dfl.loadStrucDataFromFileRegex(root, 'STRUCTURED-*.csv', daterange=daterange)
df = dfl.loadPrometheusDataFromFileRegex(root, 'securiti_appliance_', metricsArr, '.csv', daterange=daterange)
dfus = dfl.loadUnstrucDataFromFileRegex(root, 'UNSTRUCTURED-*.csv', daterange=daterange)
df = pd.concat([df, dfus, dfst, dfsp], ignore_index=True)

cat_order = [
            "dataScannedinGB",
            "numberOfColsScanned", "numberOfChunksScanned", 
             "scanTime", "fileDownloadTimeInHrs","uniqPodCount",
             "numFilesScanned", "avgFileSizeInMB", "IdleTimeInHrs",
             "cpu_used_avg", "memory_used_avg",
             "tmp_taskq_avg", "taskq_max"
             ]

title = 'Appliance plot for appliance_id '+appliance_id+' between '+fromDt+' and '+toDt
dfp = df[(df['appliance_id'] == appliance_id) 
         & (df['metrics'].isin(cat_order))
         & ((df['ts'] >= fromDt) & (df['ts'] <= toDt))
         ].drop_duplicates()
cat_order_overlap = sorted(set(cat_order).intersection(dfp.metrics.unique()),key=lambda x:cat_order.index(x))
fig  = dfl.plotMetricsFacetForApplianceId(dfp, title, {"metrics": cat_order_overlap})
fig.update_layout(plot_bgcolor="black")
fig.show()
